In [1]:
import os

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
import torch.optim as optim


from datetime import datetime
from torch import nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing, DataParallel
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

from DataClasses import lmdb_dataset, Dataset, DataListLoader
from ModelFunctions import train, evaluate, inference

In [2]:
import sys
sys.path.append(os.path.expanduser('../ocpmodels/models'))
sys.path.append(os.path.expanduser('../../ocp_airi'))

from spinconv_with_tag import spinconv

In [3]:
#вызывается каждый раз, когда датасет отдаёт элемент (систему)
#делаем из данных матрицу векторов-атомов, список рёбер (edge_index) и матрицу векторов-рёбер; надо писать свою функцию для каждой сети
def preprocessing(system):
    return system

In [4]:
#config
batch_size = 2
num_workers = 0

features_cols = ['feature_1']

target_col = 'y_relaxed'
lr = 0.001
epochs = 20

In [5]:
# #чтобы тензор по умолчанию заводился на куде
# if torch.cuda.is_available():
#     torch.set_default_tensor_type('torch.cuda.FloatTensor')
#     print('cuda')

In [6]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

cuda


In [7]:
#инициализируем тренировочный датасети и тренировочный итератор
train_dataset_file_path = os.path.expanduser("../../ocp_datasets/data/is2re/all/train/data.lmdb")

training_set = Dataset(train_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)
training_generator = DataListLoader(training_set, batch_size=batch_size)

In [8]:
#инициализируем валидационный датасет и валидационный итератор
val_dataset_file_path = os.path.expanduser("../../ocp_datasets/data/is2re/all/val_ood_both/data_mod.lmdb")

valid_set = Dataset(val_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)
valid_generator = DataListLoader(valid_set, batch_size=batch_size, num_workers=num_workers)

In [9]:
try:
    lmdb_dataset(train_dataset_file_path).describe()
except:
    pass

total entries: 460328
info for item: 0
edge_index:...............<class 'torch.Tensor'>..... [2, 2964]
pos:......................<class 'torch.Tensor'>.....   [86, 3]
cell:.....................<class 'torch.Tensor'>..... [1, 3, 3]
atomic_numbers:...........<class 'torch.Tensor'>.....      [86]
natoms:...................       <class 'int'>.....        86
cell_offsets:.............<class 'torch.Tensor'>..... [2964, 3]
force:....................<class 'torch.Tensor'>.....   [86, 3]
distances:................<class 'torch.Tensor'>.....    [2964]
fixed:....................<class 'torch.Tensor'>.....      [86]
sid:......................       <class 'int'>.....   2472718
tags:.....................<class 'torch.Tensor'>.....      [86]
y_init:...................     <class 'float'>.....    6.2825
y_relaxed:................     <class 'float'>.....   -0.0256
pos_relaxed:..............<class 'torch.Tensor'>.....   [86, 3]


In [10]:
#model
model = spinconv(None, None, 1, otf_graph=True, regress_forces=False, use_pbc=True)
model = DataParallel(model)

#optimizer and loss
optimizer = optim.AdamW(model.parameters(), lr=lr)
criterion = nn.L1Loss()

#переносим на куду если она есть
model = model.to(device)
criterion = criterion.to(device)

max num elements 54
max num elements 54
max num elements 54
max num elements 54
max num elements 54
max num elements 54


In [11]:
timestamp = str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))

print(timestamp)

2021-10-05-10-07-44


In [12]:
#tensorboard writer, при первом запуске надо руками сделать папку для логов

# server
#log_folder_path = "../../ocp_results/logs/tensorboard/out_base_model"

# colab
# log_folder_path = "/content/drive/MyDrive/ocp_results/logs/tensorboard/out_base_model"

# user_specific 
log_file_path = "../logs/tensorboard_airi"

writer = SummaryWriter(log_file_path + '/' + timestamp)

In [13]:
%%time
logfile_str = {
    "train_dataset_file_path": train_dataset_file_path,
    "val_dataset_file_path": val_dataset_file_path,
    "features_cols": features_cols,
    "target_col": target_col,
    "batch_size": batch_size,
    "num_workers": num_workers,
    "epochs": epochs,
    "lr": lr
}

#граф модели
try:
    #trace_system = dict(list(next(iter(training_generator))[0]))
    writer.add_graph(model, trace_system)
except:
    print('no graph')
writer.add_text(timestamp, str(logfile_str))

no graph
CPU times: user 450 µs, sys: 147 µs, total: 597 µs
Wall time: 483 µs


## Training

In [14]:
%%time
loss = []
loss_eval = []

print(timestamp)
print(f'Start training model {str(model)}')
for i in range(epochs):
    loss.append(train(model, training_generator, optimizer, criterion, epoch=i, writer=writer, device=device))
    loss_eval.append(evaluate(model, valid_generator, criterion, epoch=i, writer=writer, device=device))

2021-10-05-10-07-44
Start training model DataParallel(
  (module): spinconv(
    (act): Swish()
    (distance_expansion_forces): GaussianSmearing()
    (volume_G_smear): GaussianSmearing()
    (embeddingblock2): EmbeddingBlock(
      (act): Swish()
      (fc1): Linear(in_features=200, out_features=200, bias=True)
      (fc2): Linear(in_features=200, out_features=1600, bias=True)
      (fc3): Linear(in_features=200, out_features=32, bias=True)
      (source_embedding): Linear(in_features=54, out_features=32, bias=True)
      (target_embedding): Linear(in_features=54, out_features=32, bias=True)
      (embed_fc1): Linear(in_features=64, out_features=8, bias=True)
      (softmax): Softmax(dim=1)
    )
    (distfc1): Linear(in_features=200, out_features=200, bias=True)
    (distfc2): Linear(in_features=200, out_features=200, bias=True)
    (dist_block): DistanceBlock(
      (distance_expansion): GaussianSmearing()
      (fc1): Linear(in_features=200, out_features=200, bias=True)
    )
    

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/share/miniconda3/envs/ocp_models/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/share/miniconda3/envs/ocp_models/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/share/catalyst/ocp_airi/airi_utils/../../ocp_airi/ocpmodels/common/utils.py", line 103, in cls_method
    return f(self, *args, **kwargs)
  File "/share/catalyst/ocp_airi/airi_utils/../ocpmodels/models/spinconv_with_tag.py", line 265, in forward
    outputs = self._forward_helper(
  File "/share/catalyst/ocp_airi/airi_utils/../ocpmodels/models/spinconv_with_tag.py", line 302, in _forward_helper
    x = self.embeddingblock2(x, source_element, target_element, atomic_numbers, tag)
  File "/share/miniconda3/envs/ocp_models/lib/python3.8/site-packages/t